# NLP: Review Summary, Sentiment Analysis
__Done__ 
Clean contractions, Lemmatize, Remove Stop words (think specific to reviewing airbnb), parts of speech (removing pronouns, San Francisco), Tokenize  
  
__By Listing IDs__
- Vectorize  
- Topic Model (same as Dim Reduction)
- Sumy LSA (on full sentences)
- Gensim summarizer()TextBlob
  
__on Listings data__  
- Clustering on reviews: try all the methods, Kmeans, DBScan, .....
- StandardScale first

## Imports

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import dask.dataframe as dd
import pandas as pd
import numpy as np
import math
import os
import pickle
import timeit
from collections import Counter

# Graphics
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='svg'
import seaborn as sns
sns.set(font_scale=1.2)
plt.style.use('seaborn')
# from mlxtend.plotting import plot_decision_regions
# from mpl_toolkits.mplot3d import Axes3D

In [2]:
from __future__ import print_function
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from operator import itemgetter
import contractions
from langdetect import detect

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.preprocessing import StandardScaler

In [4]:
# %run '/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/notebooks/helper_functions.py' 
os.chdir('/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/notebooks')
import helper_functions
from helper_functions import clean_dolla, plot_bar, cluster_inertia#, display_topics

In [5]:
os.chdir('/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/data')
with open('listings_sf_cleaned.pkl', 'rb') as file:
    listings = pickle.load(file)

print(listings.shape)
listings.head()

(6633, 95)


,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,bathrooms,bedrooms,beds,price,weekly_price,...,neighborhood_Potrero Hill,neighborhood_Presidio Heights,neighborhood_Russian Hill,neighborhood_Seacliff,neighborhood_South of Market,neighborhood_Twin Peaks,neighborhood_Visitacion Valley,neighborhood_West of Twin Peaks,neighborhood_Western Addition,host_in_same_neighborhood
0,1,1,1,1,1,1.0,1.0,2.0,170.0,1120.0,...,0,0,0,0,0,0,0,0,1,1
1,0,2,2,1,1,1.0,2.0,3.0,235.0,1600.0,...,0,0,0,0,0,0,0,0,0,1
2,0,10,10,1,1,4.0,1.0,1.0,65.0,485.0,...,0,0,0,0,0,0,0,0,0,1
3,0,10,10,1,1,4.0,1.0,1.0,65.0,490.0,...,0,0,0,0,0,0,0,0,0,1
4,0,2,2,1,1,1.5,2.0,2.0,675.0,0.0,...,0,0,0,0,0,0,0,0,1,1


In [6]:
reviews=pd.read_csv('SF/201808/reviews_201808.csv')
reviews.columns
drop_list = ['reviewer_id', 'reviewer_name']
print(reviews.shape)
[reviews.drop(x, inplace=True, axis=1) for x in drop_list]
reviews.head()

(278884, 6)


,listing_id,id,date,comments
0,958,5977,2009-07-23,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,Holly's place was great. It was exactly what I...


In [329]:
# !pwd
# import pickle
# with open('reviews.pkl', 'wb') as file: # the orig has no pronouns removed
#     pickle.dump(reviews, file)
# with open('reviews_cleaned2.pkl', 'wb') as file: # reviews_cleaned2 is the one with the pronouns removed
#     pickle.dump(reviews_cleaned, file)
# with open('reviews_cleaned.pkl', 'rb') as file:
#     reviews_cleaned = pickle.load(file)

/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/notebooks


## Preprocess

#### Preview

In [7]:
ix=2000
print(reviews['comments'][ix])
print()
print(contractions.fix(reviews['comments'][ix]))

I had great time staying at Brenda's house. She is an exellent host, very friendly and helpful. The place was beautiful and the location perfect. I can really recommend staying with Brenda while you're in San Francisco. I only had very few days in SF but I would like to come back and stay at Brenda's place again. Thanks for everything!!

I had great time staying at Brenda's house. She is an exellent host, very friendly and helpful. The place was beautiful and the location perfect. I can really recommend staying with Brenda while you are in San Francisco. I only had very few days in SF but I would like to come back and stay at Brenda's place again. Thanks for everything!!


In [8]:
reviews.head()

,listing_id,id,date,comments
0,958,5977,2009-07-23,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,Holly's place was great. It was exactly what I...


#### Actual work

In [9]:
# Remove Stop Words
stop = stopwords.words('english')
print(len(stop))
punctuations = ['.', ',', '(', ')', "!", "?", "'", '"', "<", ">", "-", ":", ";"]
more = ["/n", "/t", "'s"]
airbnb =['me', 'us', 'our', 'you', 'your', 'yours', 'her', 'his', 'him', 'her', 'was', 'is', 'them', 'there', 
         'great', 'place', 'stay', 'recommend',  'would', 'even', 'good', 'wait', 'still'] 
stop += punctuations
print(len(stop))
stop += more
print(len(stop))
stop += airbnb
print(len(stop))

179
192
195
218


In [10]:
def clean_text(text):
    if (type(text)==float):         # For missings
        return ""
    else:
        text = contractions.fix(text) # Fix Contractions
        tokens = word_tokenize(text)  # Tokenize
        # remove pronouns
        pos = nltk.pos_tag(tokens)   
        list_to_remove=[]
        for i in range(len(tokens)):
            if len(pos[i][1])>0 and pos[i][1] in ("NNP", "NNPS"):
                list_to_remove.append(i)
        tokens = np.delete(tokens, list_to_remove)
        # Lemmatize
        lemmatize = WordNetLemmatizer()
        clean_text = [lemmatize.lemmatize(token.lower().strip(), pos='v') for token in tokens]
        # Remove stop words
        clean_text = [x for x in clean_text if x not in stop]
    
    return ' '.join(clean_text)

In [342]:
# reviews_cleaned = [clean_text(r) for r in reviews['comments']]
# print(len(reviews_cleaned))
# reviews_cleaned[22]

In [12]:
reviews_cleaned = [comment for comment in reviews_cleaned['comments'] if detect(comment) == 'en']
print(len(reviews_cleaned))
reviews_cleaned[22]

In [13]:
os.chdir('/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/data')
import pickle
# with open('reviews_cleaned.pkl', 'wb') as file: # the orig has no pronouns removed
#     pickle.dump(reviews_cleaned, file)
# with open('reviews_cleaned2.pkl', 'wb') as file: # reviews_cleaned2 is the one with the pronouns removed
#     pickle.dump(reviews_cleaned, file)
# with open('reviews_cleaned2.pkl', 'rb') as file:
#     reviews_cleaned = pickle.load(file)
with open('reviews_cleaned3.pkl', 'rb') as file:
    reviews_cleaned = pickle.load(file)

In [14]:
# Tokenize
tokenized = [word_tokenize(i) for i in reviews_cleaned]
print(len(tokenized)) #278884
print(len(tokenized[0]))
print(tokenized[0])

281273
33
['experience', 'without', 'doubt', 'five', 'star', 'experience', 'husband', 'consummate', 'host', 'friendly', 'accomodating', 'honor', 'privacy', 'apartment', 'charm', 'layout', 'full', 'view', 'access', 'home', 'garden', 'location', 'perfect', 'full', 'engagement', 'city', 'close', 'mass', 'transit', 'walk', 'proximity', 'next', 'visit']


### Pair with Listing IDs

In [235]:
# Get the list of unique listing IDs for iterating through
listing_ids = reviews['listing_id'].unique()

array([ 958, 5858, 7918, 8142, 8339])

In [221]:
# Get the reviews for one listing at a time
for i in listing_ids:
    ix_one_review = list(reviews[reviews['listing_id']==i].index)
    reviews_one_listing = [reviews_cleaned[ix] for ix in ix_one_review]
#     Need to build next step here or else it'll get overwritten without doing work

152


[0, 1, 2, 3, 4]

In [ ]:
list(reviews[reviews['listing_id']==5858].index)

## Topic Modeling - Need to split by listing ID

In [ ]:
len(reviews_sf.groupby('listing_id').count()) # 5491 listings 
reviews_sf.groupby('listing_id').count()[:3]

### All Data

In [373]:
# Vectorize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,2), stop_words=stop, lowercase=True)
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words=stop, lowercase=True)

In [156]:
def display_topics(model, fit, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic {a:} score: {b: 6.1f}%".format(a=ix, b=100*sum(fit[:,ix])/fit.sum()   ))
        else:
            print("\nTopic {a:} score: {b: 6.1f}%".format(a=topic_names[ix], b=100*sum(fit[:,ix])/fit.sum()   ))
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

#### Wrap all functions

In [374]:
# Function for topic for each individual listing
def topic_per_listing(data, n_comp=3, n_words=10):
    cv = count_vectorizer.fit_transform(data)
    tfidf = tfidf_vectorizer.fit_transform(data)
    print("Count Vectorizer Shape", cv.shape)
    print("TFIDF Vectorizer Shape", tfidf.shape)
    print()
    
    print("NMF Decomposition with Count Vectorizer")
    nmf = NMF(n_components=n_comp)
    nmf_data = nmf.fit_transform(cv)
    display_topics(nmf, nmf_data, count_vectorizer.get_feature_names(), n_words)
    print()
    
    print("LSA Decomposition with TIFIDF Vectorizer")
    lsa_tfidf = TruncatedSVD(n_components=n_comp)
    lsa_tfidf_data = lsa_tfidf.fit_transform(tfidf)
    display_topics(lsa_tfidf, lsa_tfidf_data, tfidf_vectorizer.get_feature_names(), n_words)
    print()
    
    print("LSA Decomposition with Count Vectorizer")
    lsa_cv = TruncatedSVD(n_components=n_comp)
    lsa_cv_data = lsa_cv.fit_transform(cv)
    display_topics(lsa_cv, lsa_cv_data, count_vectorizer.get_feature_names(), n_words)
    
    return nmf, nmf_data, lsa_tfidf, lsa_tfidf_data, lsa_cv, lsa_cv_data

In [375]:
nmf, nmf_data, lsa_tfidf, lsa_tfidf_data, lsa_cv, lsa_cv_data = topic_per_listing(reviews_cleaned, n_comp=3, n_words=10)

Count Vectorizer Shape (281273, 1097647)
TFIDF Vectorizer Shape (281273, 1097647)

NMF Decomposition with Count Vectorizer

Topic 0 score:   25.0%
room, nice, clean, house, bed, bathroom, really, comfortable, get, also

Topic 1 score:   41.2%
apartment, walk, location, clean, easy, need, restaurants, park, well, everything

Topic 2 score:   33.8%
home, host, make, feel, time, welcome, house, wonderful, beautiful, like

LSA Decomposition with TIFIDF Vectorizer

Topic 0 score:  102.1%
location, host, clean, nice, apartment, room, comfortable, everything, home, really

Topic 1 score:   -4.6%
location, location host, host location, location clean, perfect location, host, clean location, apartment location, location perfect, location nice

Topic 2 score:    2.4%
host cancel, cancel reservation, automate post, arrival automate, automate, days arrival, cancel, reservation, post, arrival

LSA Decomposition with Count Vectorizer

Topic 0 score:  102.3%
host, room, clean, location, apartment, ho

In [376]:
nmf4, nmf_data4, lsa_tfidf4, lsa_tfidf_data4, lsa_cv4, lsa_cv_data4 = topic_per_listing(reviews_cleaned, n_comp=4, n_words=10)

Count Vectorizer Shape (281273, 1097647)
TFIDF Vectorizer Shape (281273, 1097647)

NMF Decomposition with Count Vectorizer

Topic 0 score:   19.8%
room, nice, clean, house, bed, bathroom, really, comfortable, also, get

Topic 1 score:   29.9%
apartment, walk, restaurants, easy, park, need, get, well, also, neighborhood

Topic 2 score:   23.0%
home, make, feel, time, house, like, welcome, beautiful, city, make feel

Topic 3 score:   27.2%
host, location, clean, perfect, comfortable, wonderful, definitely, everything, friendly, helpful

LSA Decomposition with TIFIDF Vectorizer

Topic 0 score:  102.1%
location, host, clean, nice, apartment, room, comfortable, everything, home, really

Topic 1 score:   -4.6%
location, location host, host location, location clean, perfect location, host, clean location, apartment location, location perfect, location nice

Topic 2 score:    2.5%
host cancel, cancel reservation, automate post, arrival automate, automate, days arrival, cancel, reservation, pos

## Review Summarizer
SUMY:  
1. Lex Rank (google)
2. Luhn (wasn't very good)
3. LSA (PCA)
4. Text Rank (google)

In [371]:
#Plain text parsers since we are parsing through text
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [363]:
# Get the reviews for one listing at a time
for i in listing_ids[:1]:
    ix_one_review = list(reviews[reviews['listing_id']==i].index)
    reviews_one_listing = [reviews['comments'].iloc[ix] for ix in ix_one_review]

In [364]:
# Prepare the data
reviews_one_listing2 = '. '.join(reviews_one_listing)
print(len(reviews_one_listing2))
reviews_one_listing2

54974


'Our experience was, without a doubt, a five star experience. Holly and her husband, David, were the consummate hosts; friendly and accomodating while still honoring our privacy. The apartment was a charming layout with a full view and access to the home\'s garden The location is perfect for full engagement with the city; close to mass transit with walking proximity to the Haight, the Mission, the Castro and Golden Gate Park. I can\'t wait for our next visit.\r\n\r\nTed and Karen Wingerd. Returning to San Francisco is a rejuvenating thrill but this time it was enhanced by our stay at Holly and David\'s beautifully renovated and perfectly located apartment. You do not need a car to enjoy the City as everything is within walking distance - great restaurants, bars and local stores. With such amenable hosts and a place to stay that enhances one\'s holiday, we will be returning again and again.. We were very pleased with the accommodations and the friendly neighborhood. Being able to make a

In [367]:
def review_summarizer(text, num_sentences=3):
    parser = PlaintextParser(text, Tokenizer('english'))
    
    Summarizer_LexRank = LexRankSummarizer()
    summary_LexRank = Summarizer_LexRank(parser.document, num_sentences) 
    print("LexRank Summarizer")
    for sentence in summary_LexRank:
        print(sentence)
    print()
    
    Summarizer_Lsa = LsaSummarizer()
    summary_LSA = Summarizer_Lsa(parser.document, num_sentences)
    print("LSA Summarizer")
    for sentence in summary_LSA:
        print(sentence)
    print()
    
    Summarizer_Luhn = LuhnSummarizer()
    summary_Luhn = Summarizer_Luhn(parser.document, num_sentences)
    print("Luhn Summarizer")
    for sentence in summary_Luhn:
        print(sentence)
    print()
    
    Summarizer_TextRank = TextRankSummarizer()
    summary_TextRank = Summarizer_TextRank(parser.document, num_sentences)
    print("Text Rank Summarizer")
    for sentence in summary_TextRank:
        print(sentence)
    print()
    
    return Summarizer_LexRank, summary_LexRank, Summarizer_Lsa, summary_LSA, \
            Summarizer_Luhn, summary_Luhn, Summarizer_TextRank, summary_TextRank

In [368]:
LexRank, sum_LexRank, Lsa, sum_LSA, Luhn, sum_Luhn, TextRank, sum_TextRank = \
review_summarizer(reviews_one_listing2, num_sentences=2)

LexRank Summarizer
Holly's apartment was a great stay!
The location was great, lovely part of the city and a great base to explore.. We were at Holly's place for 10 days, and we enjoy it so much!!!

LSA Summarizer
Located near public transport and a very affordable Lyft ride from most places in the city, I would love to stay at Holly's again.
The bed is very comfortable.. 무엇보다 위치가 제일 마음에 들었습니다 공항과 시내를 왔다갔다 하기좋았고 늦게들어가도 밤에 무섭지 않아서 좋았습니다 마트가 가까이 있어서 너무 편리했어요!

Luhn Summarizer
We would happily stay there again and would highly recommend the place to others.. Una parte muy importante de cada viaje es el lugar donde uno se hospedará durante la estadía en el extranjero, con toda seguridad puedo decir que el apartamento de Holly Coleman nos brindó una excelente experiencia, ya que la localización de éste es ideal para alguien que desea conocer San Francisco con calma y teniendo los barrios más interesantes en una distancia muy corta.
Una parte muy importante de cada viaje es el lugar donde un

# Unused code

In [249]:
# LDA
from sklearn.decomposition import LatentDirichletAllocation
n_comp = 6
lda = LatentDirichletAllocation(n_components=n_comp, max_iter=5, learning_method='online', \
                                learning_offset=50., random_state=0).fit_transform(cv)
print(lda.shape)
lda[0]

(278884, 6)


array([0.00238384, 0.01954079, 0.95447327, 0.00238372, 0.01883486,
       0.00238352])